<a href="https://colab.research.google.com/github/pythoneerHiro/binance/blob/main/Sockets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dependencies

In [ ]:
%%capture
%%shell
pip install confluent-kafka
pip install icecream dask python-binance 

In [ ]:
from icecream import ic
import pandas as pd
import numpy as np
import logging as log

Credentials

In [ ]:
from client import client,async_client

Sockets

In [ ]:
FUTURES_DEPTH = "futures_depth" 
FUTURES_AGG = "futures_agg"
FUTURES = "futures" #not working

import websockets
import logging as log

import asyncio


class Socket():

  from typing import Callable,Awaitable


  def __init__(self, symbol:str, socket_type:str):

    self.symbol = symbol
    self.socket_type = socket_type
    self.socket = self.futures_socket_gen(symbol,socket_type)

    return

  @staticmethod
  async def handle_socket_message(msg):
      print(msg)

  @staticmethod
  def futures_socket_gen(symbol:str, socket_type:str):
      from binance import BinanceSocketManager

      bm = BinanceSocketManager(async_client)

      sockets = {
          FUTURES_DEPTH: bm.futures_depth_socket,
          FUTURES_AGG: bm.aggtrade_futures_socket,
      }
      
      # Set up the socket based on the socket type
      socket = sockets.get(socket_type,None)

      if socket == None: raise ValueError("Invalid socket type")
      
      return socket(symbol)
  
  @staticmethod
  async def start_socket(socket, *socket_handlers:Callable[[str],Awaitable[None]]):
      RETRY_INTERVAL = 20
      TIMEOUT = 60*5 #FIXME: change

      while True:
          try:
              async with socket as stream:
                  while True:
                      msg = await asyncio.wait_for(stream.recv(), timeout=TIMEOUT)
                      await asyncio.gather(*[handle_socket_message(msg) for handle_socket_message in socket_handlers])

          except websockets.exceptions.ConnectionClosed:
              log.info(f"Websocket disconnected. Retrying in {RETRY_INTERVAL} seconds.")
              await asyncio.sleep(RETRY_INTERVAL)
          except Exception as e:
              log.exception(f"Exception in start_socket: {e}")
              await asyncio.sleep(1)

  async def start(self, *socket_handlers:Callable[[str],None]): 
      socket_handlers = self.handle_socket_message if len(socket_handlers)==0 else socket_handlers
      return await self.start_socket(self.socket,socket_handlers)


In [ ]:
symbol = "BTCUSDT"
socket = Socket(symbol,FUTURES_AGG)
await socket.start()

CancelledError: ignored

In [ ]:
from dataclasses import dataclass
from typing import List


@dataclass
class ConsumerProperties():
  Topic:str 
  ConsumerGroupID:str = "python-group-1"
  ReadFrom:str = "earliest"

class Kafka():
  from confluent_kafka import Producer

  def __init__(self):
    
    self.config_file = self.Download_Kafka_Gist()

    self.config = self.Read_ccloud_config(self.config_file)

  @staticmethod
  def Download_Kafka_Gist()  -> str :
    import requests
    gist_url = "https://gist.github.com/pythoneerHiro/02b3400c9f62c278f811362d97db2d9b"
    filename = "kafka.txt"
    raw_url = f"{gist_url}/raw/{filename}"

    response = requests.get(raw_url)

    if response.status_code == 200:
        print(response.text)
        with open(filename, "w") as f:
            f.write(response.text)
    else:
        raise f"Failed to download {filename} from {gist_url}"

    return filename
  
  @staticmethod
  def Read_ccloud_config(config_file):
      conf = {}
      with open(config_file) as fh:
          for line in fh:
              line = line.strip()
              if len(line) != 0 and line[0] != "#":
                  parameter, value = line.strip().split('=', 1)
                  conf[parameter] = value.strip()
      return conf
  
  def producer(self):
    from confluent_kafka import Producer

    producer = Producer(self.config)

    return producer
  
  def consumer(self,consumer_properties:ConsumerProperties):
    from confluent_kafka import Consumer

    props = self.config
    props["group.id"] = consumer_properties.ConsumerGroupID
    props["auto.offset.reset"] = consumer_properties.ReadFrom

    consumer = Consumer(props)
    topic = consumer_properties.Topic
    consumer.subscribe([topic])

    return consumer

  def admin(self):
    from confluent_kafka.admin import AdminClient
    
    admin_client = AdminClient(self.config)
    return admin_client 

In [ ]:
from dataclasses import dataclass
from typing import List

@dataclass
class KafkaMessage:
  topic: str
  key:str
  val:str

class KafkaApp():

  @staticmethod
  def CreateTopics(topics:List[str]):
    from confluent_kafka import KafkaException
    from confluent_kafka.admin import NewTopic

    admin_client = Kafka().admin()

    num_partition = 6
    replication_factor = 3 #minimum
    
    new_topics = [NewTopic(topic, num_partitions=num_partition, replication_factor=replication_factor) for topic in topics]
    futures = admin_client.create_topics(new_topics)
    for topic, future in futures.items():
        try:
            future.result()
            print(f"Topic {topic} created")
        except KafkaException as e:
            print(f"Failed to create topic {topic}: {e}")

  
  @staticmethod
  def Produce(message:KafkaMessage=KafkaMessage("test","key","val")):
    producer = Kafka().producer()
    topic = message.topic
    key = message.key
    value = message.val
    producer.produce(topic, key=key, value=value)
    print(f"published on {topic} - {key}:{value}")

  @staticmethod
  def Consume(consumer_ppt:ConsumerProperties=ConsumerProperties("test")):
    consumer = Kafka().consumer(consumer_ppt)
    try:
        timeout = 1.0
        while True:
          msg = consumer.poll(timeout)
          if msg is not None and msg.error() is None:
            key = msg.key().decode('utf-8')
            val = msg.value().decode('utf-8')
            print("consumed-",val,key)
            
    except KeyboardInterrupt:
        pass
    finally:
        consumer.close()

NameError: ignored

In [ ]:
topics = [
    FUTURES_AGG,
    FUTURES_DEPTH,
]
KafkaApp.createTopics(topics)

# Required connection configs for Kafka producer, consumer, and admin
bootstrap.servers=pkc-lzvrd.us-west4.gcp.confluent.cloud:9092
security.protocol=SASL_SSL
sasl.mechanisms=PLAIN
sasl.username=4QWWDYSIJSOALWM6
sasl.password=lkuZ3tt3E//GU9DVnDROsvxJD7VsBVzmLA1DIRqhNisDjUj0t3JsuSCgnZ1djHg2


Failed to create topic futures_agg: KafkaError{code=TOPIC_ALREADY_EXISTS,val=36,str="Topic 'futures_agg' already exists."}
Failed to create topic futures_depth: KafkaError{code=TOPIC_ALREADY_EXISTS,val=36,str="Topic 'futures_depth' already exists."}


In [ ]:
app = KafkaApp()
topic = "test"

for i in range(10): 
  KafkaApp.Produce(KafkaMessage(topic,"key","val"))

In [ ]:
consumer_ppt = ConsumerProperties(Topic=topic)
app.Consume(consumer_ppt)

# Required connection configs for Kafka producer, consumer, and admin
bootstrap.servers=pkc-lzvrd.us-west4.gcp.confluent.cloud:9092
security.protocol=SASL_SSL
sasl.mechanisms=PLAIN
sasl.username=4QWWDYSIJSOALWM6
sasl.password=lkuZ3tt3E//GU9DVnDROsvxJD7VsBVzmLA1DIRqhNisDjUj0t3JsuSCgnZ1djHg2


consumed- {"ordertime":1497014222380,"orderid":18,"itemid":"Item_184","address":{"city":"Mountain View","state":"CA","zipcode":94041}} 18


In [ ]:
kafka = Kafka()
producer = kafka.producer()
for i in range(1000):
  topic = "test"
  key = "key"
  value = str(i)
  producer.produce("test", key=key, value=value)
  print("produced-",i)


In [ ]:
df = pd.DataFrame(client.futures_historical_klines(
    symbol='BTCUSDT',
    interval='1d',
    start_str='2021-06-01',
    # end_str='2021-06-30'
))
df.tail()

,0,1,2,3,4,5,6,7,8,9,10,11
661,1679616000000,28296.00,28440.00,26800.00,27460.90,233941.365,1679702399999,6520037473.72420,58928,112778.534,3143260183.35860,0
662,1679702400000,27447.30,27900.00,27051.00,27469.00,173702.987,1679788799999,4774342002.99440,50287,85637.969,2354384599.28370,0
663,1679788800000,27468.50,28163.00,27200.00,27973.00,183237.527,1679875199999,5087650338.84190,47524,92101.316,2558701673.08790,0
664,1679875200000,27972.50,28180.00,26350.00,27136.90,213626.657,1679961599999,5852586857.92560,59496,100606.441,2757468076.91730,0
665,1679961600000,27112.00,27250.00,26800.00,26939.00,35361.127,1680047999999,955125359.35810,10573,17506.317,472930487.86240,0


Archived

In [ ]:
# symbol = "BTCUSDT"
# socket = await futures_socket_gen(symbol,KLINE)
# await start_socket(socket,handle_socket_message)

In [ ]:
logging.basicConfig(level=logging.DEBUG)

In [ ]:
!pytest

============================= test session starts ==============================
platform linux -- Python 3.9.16, pytest-7.2.2, pluggy-1.0.0
rootdir: /content
collected 1 item                                                               

test_list.py .                                                           [100%]

============================== 1 passed in 0.01s ===============================
